In [45]:
import pandas as pd
import numpy as np
diabetes = pd.read_csv('data/diabetes_data.csv')

In [46]:
# Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их.
# Для поиска используйте все признаки в данных. Сколько записей осталось в данных?
# Удаление повторяющихся строк по всем признакам
diabetes_no_duplicates = diabetes.drop_duplicates()

# Определение количества оставшихся записей в данных
remaining_records = diabetes_no_duplicates.shape[0]

print(f"Количество записей после удаления дубликатов: {remaining_records}")

Количество записей после удаления дубликатов: 768


In [47]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [48]:
# Удаление столбца 'Gender'
diabetes_cleaned = diabetes.drop('Gender', axis=1)

# Вывод информации о новом наборе данных
print(diabetes_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.8 KB
None


In [52]:
diabetes_data = pd.read_csv('data/diabetes_data.csv', sep=',')
diabetes_data.head(5)

dupl_columns = list(diabetes_data.columns)

mask = diabetes_data.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes_data[mask]
diabetes_data = diabetes_data.drop_duplicates(subset=dupl_columns)

low_information_cols = []

#цикл по всем столбцам
for col in diabetes_data.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_data[col].nunique() / diabetes_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
diabetes_data = diabetes_data.drop(low_information_cols, axis=1)

for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']:
    diabetes_data[col] = diabetes_data[col].apply(lambda x: np.nan if x == 0 else x)
#display(diabetes_dedupped)

diabetes_data.isnull().mean().round(2).sort_values(ascending=False)

thresh = diabetes_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
diabetes_data = diabetes_data.dropna(thresh=thresh, axis=1)
diabetes_data.info()

m = diabetes_data.shape[1]
diabetes_data = diabetes_data.dropna(thresh=m-2, axis=0)
print(f'Результирующее число записей: {diabetes_data.shape[0]}')

<class 'pandas.core.frame.DataFrame'>
Index: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   763 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  768 non-null    float64
 6   Age                       768 non-null    int64  
 7   Outcome                   768 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 54.0 KB
Результирующее число записей: 761


In [53]:
# В оставшихся записях замените пропуски на медиану. 
# Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.
diabetes_data['SkinThickness'].fillna(diabetes_data['SkinThickness'].median(), inplace=True)
skin_thickness_mean = diabetes_data['SkinThickness'].mean().round(1)
print(f'Среднее значение в столбце SkinThickness: {skin_thickness_mean}')

Среднее значение в столбце SkinThickness: 29.1


In [54]:
# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
Q1 = diabetes_data['SkinThickness'].quantile(0.25)
Q3 = diabetes_data['SkinThickness'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_skin_thickness = diabetes_data[(diabetes_data['SkinThickness'] < lower_bound) | (diabetes_data['SkinThickness'] > upper_bound)]
num_outliers_skin_thickness = outliers_skin_thickness.shape[0]

print(f"Число выбросов в признаке SkinThickness: {num_outliers_skin_thickness}")

Число выбросов в признаке SkinThickness: 87


In [55]:
# Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?
mean_skin_thickness = diabetes_data['SkinThickness'].mean()
std_skin_thickness = diabetes_data['SkinThickness'].std()

# Вычисляем границы метода z-отклонения
lower_bound_z = mean_skin_thickness - 3 * std_skin_thickness
upper_bound_z = mean_skin_thickness + 3 * std_skin_thickness

# Находим выбросы
outliers_z_skin_thickness = diabetes_data[(diabetes_data['SkinThickness'] < lower_bound_z) | (diabetes_data['SkinThickness'] > upper_bound_z)]
num_outliers_z_skin_thickness = outliers_z_skin_thickness.shape[0]

print(f"Число выбросов в признаке SkinThickness по методу z-отклонения: {num_outliers_z_skin_thickness}")

Число выбросов в признаке SkinThickness по методу z-отклонения: 4


In [56]:
# Метод межквартильного размаха для исходного признака DiabetesPedigreeFunction
q1 = diabetes_data['DiabetesPedigreeFunction'].quantile(0.25)
q3 = diabetes_data['DiabetesPedigreeFunction'].quantile(0.75)
iqr = q3 - q1

lower_bound_iqr = q1 - 1.5 * iqr
upper_bound_iqr = q3 + 1.5 * iqr

outliers_iqr = diabetes_data[
    (diabetes_data['DiabetesPedigreeFunction'] < lower_bound_iqr) | 
    (diabetes_data['DiabetesPedigreeFunction'] > upper_bound_iqr)
]
num_outliers_iqr = outliers_iqr.shape[0]

# Метод выбросов в логарифмическом масштабе
diabetes_data['log_DiabetesPedigreeFunction'] = np.log(diabetes_data['DiabetesPedigreeFunction'])

q1_log = diabetes_data['log_DiabetesPedigreeFunction'].quantile(0.25)
q3_log = diabetes_data['log_DiabetesPedigreeFunction'].quantile(0.75)
iqr_log = q3_log - q1_log

lower_bound_iqr_log = q1_log - 1.5 * iqr_log
upper_bound_iqr_log = q3_log + 1.5 * iqr_log

outliers_iqr_log = diabetes_data[
    (diabetes_data['log_DiabetesPedigreeFunction'] < lower_bound_iqr_log) | 
    (diabetes_data['log_DiabetesPedigreeFunction'] > upper_bound_iqr_log)
]
num_outliers_iqr_log = outliers_iqr_log.shape[0]

# Разница между числом выбросов
difference_outliers = num_outliers_iqr - num_outliers_iqr_log
print(f"Разница между выбросами в методах IQR и логарифмическом масштабе: {difference_outliers}")


Разница между выбросами в методах IQR и логарифмическом масштабе: 29
